In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("ARCHIVED__Restaurant_Inspection_Scores__2016-2019__20250104.csv")
df.head()

,business_id,business_name,business_address,business_city,business_state,business_postal_code,business_latitude,business_longitude,business_location,business_phone_number,...,inspection_score,inspection_type,violation_id,violation_description,risk_category,Neighborhoods,SF Find Neighborhoods,Current Police Districts,Current Supervisor Districts,Analysis Neighborhoods
0,86718,Lollipot,890 Taraval St,San Francisco,CA,94116,NaN,NaN,NaN,1.415582e+10,...,45.0,Routine - Unscheduled,86718_20180522_103116,Inadequate food safety knowledge or lack of ce...,Moderate Risk,NaN,NaN,NaN,NaN,NaN
1,5827,HILLCREST ELEMENTARY SCHOOL,810 SILVER Ave,San Francisco,CA,94134,37.729016,-122.419253,POINT (-122.419253 37.729016),1.415546e+10,...,NaN,Reinspection/Followup,NaN,NaN,NaN,92.0,92.0,2.0,2.0,7.0
2,94910,Ike's Kitchen,800 Van Ness Ave,San Francisco,CA,94109,NaN,NaN,NaN,NaN,...,NaN,New Ownership - Followup,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,64667,Jasmine Rae Bakery,1890 Bryant St #309,San Francisco,CA,94110,37.763156,-122.410351,POINT (-122.410351 37.763156),NaN,...,NaN,Reinspection/Followup,NaN,NaN,NaN,53.0,53.0,3.0,2.0,20.0
4,86718,Lollipot,890 Taraval St,San Francisco,CA,94116,NaN,NaN,NaN,1.415582e+10,...,45.0,Routine - Unscheduled,86718_20180522_103154,Unclean or degraded floors walls or ceilings,Low Risk,NaN,NaN,NaN,NaN,NaN


In [3]:
# Count total numbr of rows
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53973 entries, 0 to 53972
Data columns (total 22 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   business_id                   53973 non-null  int64  
 1   business_name                 53973 non-null  object 
 2   business_address              53973 non-null  object 
 3   business_city                 53973 non-null  object 
 4   business_state                53973 non-null  object 
 5   business_postal_code          52787 non-null  object 
 6   business_latitude             27475 non-null  float64
 7   business_longitude            27475 non-null  float64
 8   business_location             27460 non-null  object 
 9   business_phone_number         17672 non-null  float64
 10  inspection_id                 53973 non-null  object 
 11  inspection_date               53973 non-null  object 
 12  inspection_score              39541 non-null  float64
 13  i

In [4]:
# Total number of unique business in the data
total_business = df['business_name'].nunique()
print(total_business)

5775


In [5]:
# Total number of unique zip codes
total_zip_codes = df['business_postal_code'].nunique()
print(total_zip_codes)

62


In [6]:
# Total number of rows for each inspection type
inspection_type_counts = df['inspection_type'].value_counts()

print(inspection_type_counts)

inspection_type
Routine - Unscheduled                39638
Reinspection/Followup                 6718
Complaint                             2399
New Ownership                         1914
New Construction                       999
Non-inspection site visit              847
New Ownership - Followup               531
Structural Inspection                  402
Complaint Reinspection/Followup        232
Foodborne Illness Investigation        203
Routine - Scheduled                     74
Special Event                            8
Administrative or Document Review        4
Multi-agency Investigation               3
Community Health Assessment              1
Name: count, dtype: int64


In [7]:
# Total number of unique inspections
total_inspections = df['inspection_id'].nunique()
print(total_inspections)

26663


In [8]:
# Total number of unique inspections per inspection type
total_inspections_per_type = df.groupby('inspection_type')['inspection_id'].nunique()
print(total_inspections_per_type.sort_values(ascending=False))

inspection_type
Routine - Unscheduled                14077
Reinspection/Followup                 6439
New Ownership                         1592
Complaint                             1458
New Construction                       994
Non-inspection site visit              811
New Ownership - Followup               499
Structural Inspection                  394
Complaint Reinspection/Followup        227
Foodborne Illness Investigation        115
Routine - Scheduled                     46
Administrative or Document Review        4
Multi-agency Investigation               3
Special Event                            3
Community Health Assessment              1
Name: inspection_id, dtype: int64


In [21]:
# Step 1: Group by `inspection_id` to get unique inspections per business
unique_inspections = df.groupby('inspection_id').first().reset_index()

# Step 2: Calculate the average inspection score and retain additional columns
business_avg_scores = unique_inspections.groupby('business_name').agg({
    'inspection_score': 'mean',  # Average inspection score
    'business_postal_code': lambda x: x.mode()[0] if len(x.mode()) > 0 else None,  # Most frequent postal code
    'business_latitude': 'first',  # Take the first latitude (assuming consistency)
    'business_longitude': 'first',  # Take the first longitude (assuming consistency)
    'business_location': 'first'   # Take the first location (assuming consistency)
}).reset_index()

# Rename for clarity
business_avg_scores.rename(columns={'inspection_score': 'average_inspection_score'}, inplace=True)

# Display the resulting DataFrame
print(business_avg_scores)

                  business_name  average_inspection_score  \
0     #1 VERJUS CAVE, #2 VERJUS                100.000000   
1             100137 Cloud Club                       NaN   
2             111 Minna Gallery                 95.333333   
3        12 Tribes Kosher Foods                 98.000000   
4                   1428 Haight                 77.000000   
...                         ...                       ...   
5770             Zutuhil Azteca                       NaN   
5771                      Zynga                 93.333333   
5772                       Zzan                 83.000000   
5773                   iNoodles                 92.666667   
5774              vive la tarte                100.000000   

     business_postal_code  business_latitude  business_longitude  \
0                   94111                NaN                 NaN   
1                   94107                NaN                 NaN   
2                   94105          37.787378         -122.39945

In [22]:
business_avg_scores.head()

,business_name,average_inspection_score,business_postal_code,business_latitude,business_longitude,business_location
0,"#1 VERJUS CAVE, #2 VERJUS",100.000000,94111,NaN,NaN,None
1,100137 Cloud Club,NaN,94107,NaN,NaN,None
2,111 Minna Gallery,95.333333,94105,37.787378,-122.399459,POINT (-122.399459 37.787378)
3,12 Tribes Kosher Foods,98.000000,94118,37.787311,-122.446839,POINT (-122.446839 37.787311)
4,1428 Haight,77.000000,94117,NaN,NaN,None


In [23]:
# Drop rows with missing values in the specified columns
columns_to_check = ['average_inspection_score', 'business_postal_code', 'business_latitude', 'business_longitude']
business_avg_scores = business_avg_scores.dropna(subset=columns_to_check)

# Display the resulting DataFrame
print(business_avg_scores)

               business_name  average_inspection_score business_postal_code  \
2          111 Minna Gallery                 95.333333                94105   
3     12 Tribes Kosher Foods                 98.000000                94118   
5                  15 Romolo                 88.000000                94133   
9          17th & Noe Market                 96.000000                94114   
10                18 Reasons                 95.000000                94110   
...                      ...                       ...                  ...   
5764               Zero Zero                 91.000000                94107   
5766  Ziggy's Burger & Grill                 86.333333                94111   
5767                 Zingari                 93.500000                94102   
5769                  Zpizza                 93.333333                94103   
5771                   Zynga                 93.333333                94103   

      business_latitude  business_longitude        

In [25]:
business_avg_scores.to_csv("clean_data_with_locations.csv", index=False)